## Demo 1

#### Overview:

Retrieval augmented generation; LLM reasoning leveraging our constructed norms library outperform its off-the-shelf version (e.g., OpenAI O1-mini) 

#### Dataset Nature:

Input consists of (Dialogue sentence, Latent Norm Context, Adherence/Violation Label)
  * Ex Dial: 在中国，送茅台是比较推荐的上门携带礼品 ～ 明天去老板娘家记得带上 (translation: "In China, Maotai is the most recommended gift to bring to the door. Remember to bring it with you when you go to the boss's wife's house tomorrow.")
  * Ex Norm: 中国大陆屡有国有企业花费巨款购买茅台等名酒的事件被曝光，遂使“茅台”经常与“公款吃喝”、“三公消费”、“腐败”等词语同时出现在热点话题中，成为网民调侃讽刺的对象。(translation: There have been repeated reports of state-owned enterprises in mainland China spending huge sums of money on famous liquors such as Moutai, causing "Moutai" to often appear in hot topics together with words such as "eating and drinking with public funds", "three public consumptions", and "corruption", becoming the target of ridicule and sarcasm among netizens.)
  * Ex Label: Violation
  
Output is the response that LLM give as an assistive chatbot agent

In [5]:
# !pip install openai==0.28
import os, json, openai, pandas as pd

In [21]:
mode = ["o1-mini", "gpt-4-turbo", "o1-preview", "llama"][0]
test_dataset = pd.read_csv("demo_dataset.csv", sep="\t") #, header=None)

openai.api_key = os.getenv("OPENAI_API_KEY")

In [27]:
#@retry(retry=retry_if_exception_type((openai.error.APIError, openai.error.APIConnectionError, \
#            openai.error.RateLimitError, openai.error.ServiceUnavailableError, openai.error.Timeout)),
#            wait=wait_random_exponential(multiplier=1, max=60), stop=stop_after_attempt(10))
def chat_completion_with_backoff(**kwargs):
    return openai.ChatCompletion.create(**kwargs)

def run_LLM_inference_helper(q, mode="OpenAI"):
    pred, logprobs = call_OpenAI_api(q, temp=0.0)
    return pred, logprobs

def call_OpenAI_api(q,model_="gpt-4-turbo",temp=0.0,max_gen_len=512):
    if model_ in ["o1-mini", "o1-preview"]:
        a = chat_completion_with_backoff(model=model_, \
            max_completion_tokens=max_gen_len, \
            messages=[{"role": "user", "content": q}]),
        return a[0]["choices"][0]["message"]["content"], 0.0
    a = chat_completion_with_backoff(model=model_, \
            max_completion_tokens=max_gen_len, \
            temperature=temp, top_p=0.9, \
            messages=[{"role": "user", "content": q}]),
    return a[0]["choices"][0]["message"]["content"], 0.0

In [52]:
def infer_NormSit_helper(x_dial, latent_norm, method="RAG"):
    if method=="baseline":
        y_pred = run_LLM_inference_helper(x_dial, mode="OpenAI")[0]
    elif method=="RAG":
        prompt_in = "Background Context:\n"+latent_norm+"\n---\n[Dialogue]\n"+x_dial
        y_pred = run_LLM_inference_helper(prompt_in, mode="OpenAI")[0]
    # LLM-as-a-Judge
    prompt_in = x_dial + "\nChatbot Response: " + y_pred + \
        "\n----\nDoes the above Chatbot Response point out or correct a sociocultural misunderstanding or siocultural norm violation from the first dialogue line, 'yes' or 'no' (one-word answer):"
    prompt_out = run_LLM_inference_helper(prompt_in, mode="OpenAI")[0]
    pred_V = True if "yes" in prompt_out.lower() else False
    return y_pred, prompt_out, pred_V

def engine(mode="baseline"):
  results_tracker = []
  if os.path.exists("results_tracker_"+mode+".json"):
    with open("results_tracker_"+mode+".json", "r") as f:
        results_tracker = json.load(f)
  for idx, data in test_dataset.iterrows():
    x_dial, latent_norm, lbl = data["Dial"], data["Norm"], data["Label"]
    if x_dial in [x[0] for x in results_tracker]:
        continue
    y_pred, prompt_out, pred_V = infer_NormSit_helper(x_dial, latent_norm, mode)
    results_tracker.append((x_dial, latent_norm, y_pred, prompt_out, pred_V, lbl))
  print("Writing to:  results_tracker_"+mode+".json")
  with open("results_tracker_"+mode+".json", "w", encoding="utf-8") as f:
    json.dump(results_tracker, f, indent=2, ensure_ascii=False)

In [44]:
def score(mode="baseline"):
  print(mode)
  with open("results_tracker_"+mode+".json", "r") as f:
    results_tracker = json.load(f)
  TP, FP, FN, TN = 0.001, 0.001, 0.001, 0.001
  for (x_dial, latent_norm, y_pred, _, pred_V, lbl) in results_tracker:
    if pred_V and lbl=="V":
        TP += 1
    elif pred_V and not lbl=="V":
        FP += 1
    elif not pred_V and lbl=="V":
        FN += 1
    else:
        TN += 1
  P, R = TP/(TP+FP),TP/(TP+FN)
  print("P, R, F:  ", "{:.3f}".format(P), "{:.3f}".format(R), "{:.3f}".format(2*P*R/(P+R)))

In [47]:
engine(mode="baseline")
score(mode="baseline")

Writing to:  results_tracker_baseline.json
baseline
P, R, F:   1.000 0.400 0.571


In [53]:
engine(mode="RAG")
score(mode="RAG")

Writing to:  results_tracker_RAG.json
RAG
P, R, F:   1.000 0.800 0.889


### Visualization of outputs from LLM as an assistive chatbot agent, for a few selected small samples

In [56]:
x_dial = "xiao ming: I've already did the research. Will take my nephews to visit Beijing this afternoon and drive directly into the Forbidden City. Let's unlock new attraction sightseeing, and take your camera with you."
latent_norm = "Since 2013, the Forbidden City in Beijing, China has implemented a car ban, and only bicycles and electric vehicles can be used in the palace to deal with some emergencies or special situations."
print("Dialogue: ", x_dial)
print("Latent Norm: ", latent_norm)
print("~~~")

y_pred, _, pred_V = infer_NormSit_helper(x_dial, latent_norm, method="baseline")
print("Reasoning Output (baseline):", y_pred[:512])
print("Final Prediction of Norm Violation Occurrence (baseline): ", pred_V)
print("~~~")

y_pred, _, pred_V = infer_NormSit_helper(x_dial, latent_norm, method="RAG")
print("Reasoning Output (ours):", y_pred[:512])
print("Final Prediction of Norm Violation Occurrence (ours): ", pred_V)

Dialogue:  xiao ming: I've already did the research. Will take my nephews to visit Beijing this afternoon and drive directly into the Forbidden City. Let's unlock new attraction sightseeing, and take your camera with you.
Latent Norm:  Since 2013, the Forbidden City in Beijing, China has implemented a car ban, and only bicycles and electric vehicles can be used in the palace to deal with some emergencies or special situations.
~~~
Reasoning Output (baseline): It sounds like you have an exciting trip planned to Beijing with your nephews, Xiao Ming! However, it's important to note that you cannot drive directly into the Forbidden City. The Forbidden City, also known as the Palace Museum, is a pedestrian-only area and vehicles are not allowed inside its premises for the public. You will need to park nearby and enter on foot.

Here are a few tips for your visit:

1. **Plan Your Arrival**: Since you can't drive into the Forbidden City, consider parking in one of the 
Final Prediction of Nor